In [26]:
import tensorflow as tf
from keras import layers, models, callbacks
import numpy as np
import matplotlib.pyplot as plt

# --- Data Loading & Preprocessing ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1,28,28,1).astype('float32') / 255.0
x_test  = x_test.reshape(-1,28,28,1).astype('float32') / 255.0
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=10)

# --- CNN Classifier Model ---
def build_cnn():
    model = tf.keras.Sequential([
        layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),  

        layers.Dense(10, activation='softmax')
    ])
    return model

# --- Compile and Train CNN ---
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.95, nesterov=True)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.01 * 0.95**epoch, verbose=0)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

cnn_model = build_cnn()
cnn_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
history = cnn_model.fit(x_train, y_train_oh, epochs=44, batch_size=512, callbacks=[lr_schedule, early_stop])

# --- Evaluate CNN ---
test_loss, test_acc = cnn_model.evaluate(x_test, y_test_oh)
print(f"\n✅ CNN Test Accuracy: {test_acc * 100:.2f}%")

# --- Simple Generator ---
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, input_dim=100),
        layers.LeakyReLU(0.2),
        layers.Dense(28*28, activation='tanh'),
        layers.Reshape((28, 28))
    ])
    return model

generator = build_generator()

# --- Generate a Sample Image ---
np.random.seed(42)
tf.random.set_seed(42)
noise = np.random.normal(0, 1, (1, 100))
generated_image = generator.predict(noise, verbose=0)[0]
scaled_image = (generated_image + 1) / 2.0  # Rescale from [-1, 1] to [0, 1]

# --- Plot the Generated Image ---
plt.imshow(scaled_image, cmap='gray')
plt.axis('off')
plt.title("🖼 Generated Black and White Image")
plt.show()

Epoch 1/44
 96/118 [=======================>......] - ETA: 0s - loss: 0.8633 - accuracy: 0.9060

KeyboardInterrupt: 

In [ ]:
# --- Predict the Digit from the Generated Image using CNN ---
generated_image_reshaped = scaled_image.reshape(1, 28, 28, 1).astype('float32')
prediction = cnn_model.predict(generated_image_reshaped, verbose=0)
predicted_label = np.argmax(prediction)

print(f"🔢 Predicted Digit by CNN: {predicted_label}")

🔢 Predicted Digit by CNN: 8
